# 2 - Training a Model on Remote Compute

In this notebook, you will train a model on remote compute powered by Azure Machine Learning. While we will be working primarily in this Jupyter notebook, the actual training will be done on a separate compute cluster.

## Imports

We first need to import several modules from the Azure Machine Learning Python SDK:

In [1]:
import os
import urllib
import shutil
import azureml
import azureml.core
from azureml.core import Workspace, Run
from azureml.core import Experiment
from azureml.core.compute import AmlCompute
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.dnn import TensorFlow
from azureml.widgets import RunDetails
from azureml.opendatasets import MNIST
from azureml.core import dataset
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.33.0


## Workspace

If you are running this notebook as a part of your Azure Machine Learning Studio workspace, you can get a reference to your workspace by simply running the following code:

In [2]:
ws = Workspace.from_config()
print("Azure ML Workspace")
print(f'Name: {ws.name}')
print(f'Location: {ws.location}')
print(f'Resource Group: {ws.resource_group}')

Azure ML Workspace
Name: carlartest
Location: westeurope
Resource Group: networkwatcherrg


## Provisioning Compute Resources

We will include the logic from the previous module around provisioning compute resources.  This will create a cluster of Standard NC6 instances that can scale from 0 to 4 nodes.

In [3]:
# Create a name for our new cluster
cpu_cluster_name = 'Std-V2-Cluster'

# Verify that cluster does not exist already
try:
    cpu_cluster = AmlCompute(workspace=ws, name=cpu_cluster_name)
    print('Cluster already exists.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6',
                                                           max_nodes=4)
    cpu_cluster = AmlCompute.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Cluster already exists.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Create an Experiment

In [4]:
# Get reference to the workspace
ws = Workspace.from_config()
print("Azure ML Workspace")
print(f'Name: {ws.name}')
print(f'Location: {ws.location}')
print(f'Resource Group: {ws.resource_group}')

# Create an experiment, or get a reference to the experiment if it already exists
experiment_name = 'cutie'
exp = Experiment(workspace=ws, name=experiment_name)
print("Azure ML Experiment")
print(f'ID: {exp.id}')
print(f'Name: {exp.name}')

Azure ML Workspace
Name: carlartest
Location: westeurope
Resource Group: networkwatcherrg
Azure ML Experiment
ID: cf203022-9828-45f5-971e-34425bfa0826
Name: cutie


Next, we will create a directory that will hold the script we will use for training our model.  This directory will eventually be sent to our remote compute cluster:

In [5]:
import os
script_folder = os.path.join(os.getcwd(), "cutie")
os.makedirs(script_folder, exist_ok=True)

Next, we need to output the script that we will use to train our model.  It will have the name `train.py` in our `keras-mnist` directory.

Within an experiment in Azure Machine Learning, each time you execute a test on your experiment, it is considered a `Run`.  The documentation defines this as:

> A run represents a single trial of an experiment. Runs are used to monitor the asynchronous execution of a trial, log metrics and store output of the trial, and to analyze results and access artifacts generated by the trial. [Run Class](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py)

We will need to get the context of the current `Run` from within our training script. We can use this to register metrics from our test and make them associated with that specific `Run`.

Since our training script will also need to access our `load_data` function, we need to be sure to copy the `utils.py` file into this directory:

In [6]:
import distutils.dir_util
distutils.dir_util.copy_tree('Codebase', script_folder)

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.amlignore',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.amlignore.amltmp',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.gitignore',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.idea/CUTIE.iml',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.idea/misc.xml',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.idea/modules.xml',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.idea/vcs.xml',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cutie/.idea/workspace.xml',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/carlartest/code/Users/fiabebe/CutieTest/cut

#Create a custom environment

In [7]:
tf_env = Environment.from_conda_specification(name='tensorflow-2.2-gpu', file_path='./conda_dependencies.yml')

# Specify a GPU base image
tf_env.docker.enabled = True
tf_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


## Create a ScriptRunConfig

Create a ScriptRunConfig object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on. Any arguments to your training script will be passed via command line if specified in the arguments parameter.

In [15]:
from azureml.core import ScriptRunConfig

args = ['--load_dict', True]

src = ScriptRunConfig(source_directory=script_folder,
                      script='main_train_json.py',
                      arguments=args,
                      compute_target=cpu_cluster,
                      environment=tf_env)

## Submit a run
The Run object provides the interface to the run history while the job is running and after it has completed.

In [16]:
run = Experiment(workspace=ws, name='CUTIE-Run').submit(src)
run.wait_for_completion(show_output=True)

RunId: CUTIE-Run_1631964857_d0d624f8
Web View: https://ml.azure.com/runs/CUTIE-Run_1631964857_d0d624f8?wsid=/subscriptions/3b861ce0-dd31-4dff-8de6-79240859c730/resourcegroups/networkwatcherrg/workspaces/carlartest&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/20_image_build_log.txt

2021/09/18 11:34:23 Downloading source code...
2021/09/18 11:34:25 Finished downloading source code
2021/09/18 11:34:26 Creating Docker network: acb_default_network, driver: 'bridge'
2021/09/18 11:34:26 Successfully set up Docker network: acb_default_network
2021/09/18 11:34:26 Setting up Docker configuration...
2021/09/18 11:34:27 Successfully set up Docker configuration
2021/09/18 11:34:27 Logging in to registry: carlartest.azurecr.io
2021/09/18 11:34:28 Successfully logged into carlartest.azurecr.io
2021/09/18 11:34:28 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/09/18 11:34:28 Scanning for dependencies...
2021/09/18 11:3

## Working with Estimators

To submit our training script to our compute cluster, we need to create an `Estimator` instance.  The documentation defines an `Estimator` as:

> To facilitate deep learning model training, the Azure Machine Learning Python SDK provides an alternative higher-level abstraction, the estimator class, which allows users to easily construct run configurations. You can create and use a generic Estimator to submit training script using any learning framework you choose (such as scikit-learn) on any compute target you choose, whether it's your local machine, a single VM in Azure, or a GPU cluster in Azure. For PyTorch, TensorFlow and Chainer tasks, Azure Machine Learning also provides respective PyTorch, TensorFlow, and Chainer estimators to simplify using these frameworks. [Train models with Azure Machine Learning using estimator](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-train-ml-models)

We will specifically be leveraging the `TensorFlow` Estimator class [see documentation](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.dnn.tensorflow?view=azure-ml-py) since we are leveraging Keras on top of TensorFlow.

To create our estimator, we will need the following:

1. Environment Configuration
1. Script Parameters
1. Script directory and entry point
1. Compute cluster and configuration

### Script Parameters

We will need to include two different parameters in our script parameters: the data folder, and the number of epochs for model compilation.  We can include any parameters here, and we can then leverage these to gauge model effectiveness with different parameter customizations:

In [8]:

script_params = {
    # to mount files referenced by mnist dataset
    #'--data-folder': mnist_file_dataset.as_named_input('mnist_opendataset').as_mount(),
    #'--epochs': 10
}

### Remaining Estimator Configuration

We will include our compute cluster plus its configuration as well as the script directory and entry point as we create our estimator instance:

In [10]:
cpu_cluster = AmlCompute(workspace=ws, name=cpu_cluster_name)

est = TensorFlow(source_directory=script_folder,
                 entry_script='main_train_json.py',
                 script_params=script_params,
                 compute_target=cpu_cluster,
                 framework_version='2.0',
                 pip_packages=['azureml-dataprep[fuse,pandas]','keras==2.2.4'],
                 use_gpu=True)

'TensorFlow' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or one of the Azure ML TensorFlow curated environments.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


### Submitting our First Run

We can then submit an Estimator experiment which will create a instance of the `Run` class:

In [11]:
run = exp.submit(config=est)
run

Run(Experiment: cutie,
Id: cutie_1631963595_e069edfb,
Type: azureml.scriptrun,
Status: Preparing)

Since this will take some time to complete, we can use an included utility to see the progress within our Jupyter notebook:

In [13]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

Once it completes, we can extract the metrics from this run:

In [ ]:
print(run.get_metrics())

## Registering Model

Finally, we can register our model in our workspace, so that it can be leveraged elsewhere.  We will use this model in the next clip to deploy it as a web service.

In [ ]:
# register model 
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = run.register_model(model_name='keras-mnist', 
                           model_path='outputs/model/mnist.h5',
                           model_framework=Model.Framework.TENSORFLOW,
                           model_framework_version='2.0',
                           resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5))